In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import networkx as nx
from contextily import add_basemap
from libpysal import weights
from shapely.geometry import LineString
import pandas as pd
import numpy as np
from matplotlib import cm
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import matplotlib as mpl
import os
from itertools import combinations
from scipy.sparse import csr_matrix
from scipy import linalg
from closeness_library import closeness_function
from degree_library import degree_function
from diameter_library import diameter_function

ModuleNotFoundError: No module named 'closeness_library'

In [ ]:
bikes = gpd.read_file(r'C:\\Users\\camperom\\safety-of-bike-networks\\Code\\OSM_extract\\GovermentData\\velo_mobilites_m.geojson')
bikes = momepy.extend_lines(bikes, 0.00001)
bikes.geometry = momepy.close_gaps(bikes, 0.00001)
df1 = pd.DataFrame(bikes)
list(bikes['geometry'][0].coords)
#bikes.plot(figsize=(10, 10)).set_axis_off()
#bikes = momepy.extend_lines(bikes, 0.001)
#bikes_e = momepy.extend_lines(bikes, 0.0000)
#bikes_extended.plot(figsize=(10, 10)).set_axis_off()
#bikes_e.geometry = momepy.close_gaps(bikes_e, 0.000)
#bikes_e = momepy.remove_false_nodes(bikes)
#bikes = momepy.extend_lines(bikes, 1)
def coords(geom):
    return list(geom.coords)
coords = bikes.apply(lambda row: coords(row.geometry), axis=1)
coordn = coords.to_numpy()
pandadata = pd.DataFrame(bikes)
numpydata = pandadata.to_numpy()
final_points = []
G = nx.Graph()
c = 0
dw = {}
for i in coordn:
    if i != []:
        final_points.append(i[0])
        final_points.append(i[len(i)-1])
        for j in range(len(i)-1):
            if (numpydata[c][4] == 'chronovelo'):
                G.add_edge(i[j],i[j+1], weight = 4)
                dw[(i[j],i[j+1])] = 4
                dw[(i[j + 1], i[j])] = 4
            if (numpydata[c][4] == 'veloseparatif'):
                G.add_edge(i[j],i[j+1], weight = 3)
                dw[(i[j], i[j + 1])] = 3
                dw[(i[j + 1], i[j])] = 3
            if (numpydata[c][4] == 'veloconseille'):
                G.add_edge(i[j],i[j+1], weight = 2)
                dw[(i[j], i[j + 1])] = 2
                dw[(i[j + 1], i[j])] = 2
            if (numpydata[c][4] == 'velodifficile'):
                G.add_edge(i[j],i[j+1], weight = 1)
                dw[(i[j], i[j + 1])] = 1
                dw[(i[j + 1], i[j])] = 1
        c = c + 1
# dw = duplicate_d(dw)
for i in list(G.nodes):
    nh = [n for n in G.neighbors(i)]
    if ((len(nh) == 2) and (i not in final_points)):
        G.add_edge(nh[0], nh[1], weight = dw[(i,nh[0])])
        G.remove_node(i)
for e in list(G.edges):
    if (e[0] == e[1]):
        G.remove_edge(*e)
pos = {n: [n[0], n[1]] for n in list(G.nodes)}